This notebook was written on colab, it doesn't require CUDA or CUDNN.
This notebook is used to perform object detection using trained YOLO models. It uses openCV to do this.

## Important  file and folder locations

In [ ]:
# Path to folder containing all test-images
Image_folder_path = '/content/drive/MyDrive/Computer_Vision/Yolo_builds/Helmet_detection/Test_images'

# Pre-trained Yolo weights and config file
Weight_file = '/content/drive/MyDrive/Computer_Vision/Yolo_builds/Helmet_detection/Weights_save/yolov3-voc_4500.weights' #Path to weight file
Conf_file = '/content/drive/MyDrive/Computer_Vision/Yolo_builds/Helmet_detection/yolov3-voc.cfg' #Path to conf file

# Path of TXT file containing names of classes on which the model was trained
Class_file = '/content/drive/MyDrive/Computer_Vision/Yolo_builds/Helmet_detection/class_names.txt'

# Directory where the detected images must be put
Output_dir = '/content/drive/MyDrive/Computer_Vision/Yolo_builds/Helmet_detection/Detected_images/YoloV3'

## Installations and Importations

In [ ]:
#!pip install GitPython
#Repo.clone_from("https://github.com/pjreddie/darknet", "/sample_data/darknet")

import cv2 as cv 
import numpy as np
from matplotlib import pyplot as plt
import os
from google.colab.patches import cv2_imshow  
import time

In [ ]:
# MOUNT DRIVE - (Mounted drive can be accessed at /content/drive/MyDrive)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Get the image files

In [ ]:
# Get all images files names from the image folder
IMAGE_FILES = os.listdir(Image_folder_path)
print('No of images = ',len(IMAGE_FILES))
print(IMAGE_FILES)


# The image parameters to be considered for all images in the observed data
Width = 416
Height = 416
scale = 1/255

# Store all images into a list(also display them)
IMAGES = []
for filename in IMAGE_FILES:
    image = cv.imread(Image_folder_path + '/' + filename)
    IMAGES.append(cv.resize(image, (Width, Height)))

No of images =  100
['hard_hat_workers900.png', 'hard_hat_workers903.png', 'hard_hat_workers904.png', 'hard_hat_workers902.png', 'hard_hat_workers901.png', 'hard_hat_workers947.png', 'hard_hat_workers934.png', 'hard_hat_workers939.png', 'hard_hat_workers949.png', 'hard_hat_workers913.png', 'hard_hat_workers909.png', 'hard_hat_workers917.png', 'hard_hat_workers950.png', 'hard_hat_workers952.png', 'hard_hat_workers942.png', 'hard_hat_workers915.png', 'hard_hat_workers971.png', 'hard_hat_workers922.png', 'hard_hat_workers919.png', 'hard_hat_workers930.png', 'hard_hat_workers912.png', 'hard_hat_workers967.png', 'hard_hat_workers910.png', 'hard_hat_workers931.png', 'hard_hat_workers935.png', 'hard_hat_workers927.png', 'hard_hat_workers961.png', 'hard_hat_workers914.png', 'hard_hat_workers944.png', 'hard_hat_workers962.png', 'hard_hat_workers956.png', 'hard_hat_workers943.png', 'hard_hat_workers965.png', 'hard_hat_workers921.png', 'hard_hat_workers957.png', 'hard_hat_workers918.png', 'hard_h

# The model is loaded here

In [ ]:
# read class names from text file
classes = None
with open(Class_file, 'r') as f:
    classes = [line.strip() for line in f.readlines()]
print(classes)

# generate different colors for different classes 
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

net = cv.dnn.readNet(Conf_file, Weight_file)

# create input blob 
blob = cv.dnn.blobFromImages(IMAGES, scale, (Width,Height), (0,0,0), True, crop=False)

# set input blob for the network
net.setInput(blob)

['person', 'head', 'helmet']
0.5716800689697266


## Some useful functions 

In [ ]:
# function to get the output layer names in the architecture
def get_output_layers(net):
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

# function to draw bounding box on the detected object with class name
def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h):
    label = str(classes[class_id])+', '+str(round(confidence, 3))
    color = COLORS[class_id]
    cv.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)
    #cv.putText(img, label, (x-10,y-10), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

## The detection happens here 

In [ ]:
## # Run inference through the network and gather predictions from output layers
start = time.time()
outs = net.forward(get_output_layers(net))
end = time.time()
print(end-start)

# Bounding box prediction Thresholds
conf_threshold = 0.3
nms_threshold = 0.3

# For each image output
for output_no in range(len(IMAGES)):
    image = np.copy(IMAGES[output_no]) #Get the input image to be detected on
    class_ids = [] #Stores the passing bounding box class-id
    confidences = [] #Stores the passing bounding box confidence
    boxes = [] #Stores the passing bounding box [x, y, w, h]

    # For each Scale output in each image output
    for Scale_prediction in outs:

        # No of detection box predictions = (number of anchor boxes per scale)*(width/32)*(height/32)*(scale number)^2
        # For each detection box prediction take up or ignore the bounding box based on confidence value
        for detection in Scale_prediction[output_no]:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf_threshold:          
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])
    
    # Apply non-max suppression
    indices = cv.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

    # Go through the detections remaining after nms and draw bounding box
    for i in indices:
        i = i[0]
        box = boxes[i]
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
        draw_bounding_box(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))

    # display output image 
    cv2_imshow(image) #Image must be given in BGR format to cv's imshow
    #cv.imwrite(Output_dir+IMAGE_FILES[output_no], image)